In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from glasses.nn.models.classification.efficientnet import *

x = torch.randn(1,3,224,224)
EfficientNetLite.efficientnet_lite0(mode='same')(x).shape

torch.Size([1, 1000])

In [3]:
EfficientNetLayer(32, 64)

ResNetLayer(
  (block): Sequential(
    (0): EfficientNetBasicBlock(
      (block): Sequential(
        (exp): ConvBnAct(
          (conv): Conv2dPad(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (depth): ConvBnAct(
          (conv): Conv2dPad(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=192, bias=False)
          (bn): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): SiLU()
        )
        (att): ChannelSE(
          (avg_pool): AdaptiveAvgPool2d(output_size=1)
          (att): Sequential(
            (conv1): Conv2d(192, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU()
            (conv2): Conv2d(8, 192, kernel_size=(1, 1), stride=(1, 1))
            (act2): Sigmoid()
          )
        )
        (point): Sequential(
          (0): ConvBnAct(
     

In [13]:
from glasses.nn.models import UNet
from glasses.nn.models.classification.resnet import ResNetEncoder, ResNet

UNet(encoder=lambda x: ResNet.resnet26(in_channels=1).encoder).summary((1,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           3,136
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
         Conv2dPad-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
        Conv2dPad-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
        Conv2dPad-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(62118338), tensor(62118338), tensor(236.9626), tensor(549.4333))

# Benchmark

In [ ]:
from glasses.nn.models.classification.efficientnet import *
import timm

from transfer_weights import clone_model
from benchmark import benchmark

src = timm.create_model('tf_efficientnet_lite1', pretrained='True')
dst = EfficientNetLite.efficientnet_lite1(mode='same')

In [ ]:
dst = clone_model(src, dst)

In [ ]:
dst.summary()

In [ ]:
dst

In [ ]:
import torch

transform = dst.configs['efficientnet_lite1'].transform

benchmark(dst.cuda(), transform)

In [ ]:
benchmark(src.cuda(), transform)

In [ ]:
src = timm.create_model('efficientnet_b0', pretrained='True')
src

In [ ]:
dst.summary()